In [17]:
%matplotlib widget
%load_ext autoreload

%autoreload 2

import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from helper.laax_gps import get_all_laax_slopes

slopes = get_all_laax_slopes(force=False)

In [9]:
from bokeh.io import output_notebook, show

output_notebook()
from helper.plots import plot_ski_slopes

show(plot_ski_slopes(slopes))

Loading BokehJS ...

In [13]:
from helper.laax_gps import get_gps_track
from helper.plots import plot_gps_track, plot_gps_altitude_distance
from bokeh.models import ColumnDataSource
from bokeh.layouts import gridplot

track = get_gps_track('./data/Laax_fri_Nachmittag.gpx')
slope_map = plot_ski_slopes(slopes, color_map={'intermediate': 'lightblue', 'easy': 'lightblue', 'freeride': 'lightblue', 'advanced': 'lightblue'}, show_altitude=False)
gps_source = ColumnDataSource(track)

p1 = plot_gps_track(gps_source, color='tomato', overlay_onto=slope_map)
p2 = plot_gps_altitude_distance(gps_source, color='tomato')
show(gridplot([[p1, p2]]))

In [14]:
from numpy.linalg import norm

def find_rides(height):
    diff = height.diff()
    height_diff = diff.rolling(20).mean().rolling(10).sum()
    return height_diff < -1

def find_closest(p):
    diff = all_geo_points - p
    diff = norm(diff, axis=1)
    return all_geo[np.argmin(diff)][0]

def find_closest_gps_track(track):
    closest = track.apply(lambda x: find_closest((x['la'] , x['lo'])), axis=1)
    return closest.rolling(30).median()


In [16]:
plt.figure()
test.astype(float).plot()

NameError: name 'plt' is not defined

In [ ]:
all_geo = []
for s in slopes.values():
    for p in s['geo']:
        all_geo.append((s['id'], p))
all_geo_points = np.vstack([p[1] for p in all_geo])

In [ ]:
track[find_rides(track['el'])]

In [ ]:
plt.figure()
find_rides(track['el']).astype(int).plot()
(track['el'] / track['el'].mean()).plot()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(gps_source)

In [ ]:
all_geo = []
for s in slopes.values():
    for p in s['geo']:
        all_geo.append((s['id'], p))
all_geo_points = np.vstack([p[1] for p in all_geo])

In [ ]:
all_geo_points

# Data

In [ ]:
import pandas as pd

data = pd.read_csv('./laax/drive-download-20190406T125400Z-001/NilsPodX-5CF4_20190405_1344.csv', header=0)


In [ ]:
def calc_height(baro, T, ref_baro, ref_height):
    R = 8.3144
    g_0 = 9.81
    M = 0.029
    T += 273
    return -np.log(baro/ref_baro) * R * T/ (g_0 * M) + ref_height
    

In [ ]:
ref_baro = data[['baro']].loc[0]
#Run was started at Galaaxy
ref_height = 2250 #m
T = 0 # deg C

In [ ]:
calc_height(data[['baro']], T, ref_baro, ref_height).plot()


In [ ]:
top = 2563
bottom = 2173

In [ ]:
acc = ['acc_' + i for i in 'xyz']
gyro = ['gyro_' + i for i in 'xyz']

In [ ]:
fig, axs = plt.subplots(2, sharex=True)

data[acc].plot(ax=axs[0])
calc_height(data[['baro']], T, ref_baro, ref_height).plot(ax=axs[1])

In [ ]:
points = gpx.tracks[0].segments[0].points
geo_track = np.array([(p.latitude, p.longitude, p.elevation, (p.time - points[0].time).total_seconds()) for p in points])

In [ ]:
np.cumsum(calc_distance_from_geo_series(geo_track[:, :2]))

In [ ]:
plt.figure()
plt.plot(geo_track[:,-1], geo_track[:, -2]  - geo_track[10,-2] + height['baro'].iloc[0])
heigth_sec = height.copy()
# heigth_sec.index /= 204.8
# heigth_sec.plot(ax=plt.gca())
ax2 = plt.gca().twinx()
ax2.plot(geo_track[:,-1], np.hstack([[0], np.cumsum(calc_distance_from_geo_series(geo_track[:, :2]))]))

In [ ]:
plt.figure()
plt.plot(*np.array([(p.latitude, p.longitude) for p in gpx.tracks[0].segments[0].points]).T[::-1])

In [ ]:
from numpy.linalg import norm
closest = []

for p in geo_track[:, :2]:
    diff = all_geo_points - p
    diff = norm(diff, axis=1)
    all_geo[np.argmin(diff)]
    closest.append(all_geo[np.argmin(diff)])

tracks = pd.Series([p[0] for p in closest])
difficulty = [slopes[int(t)]['difficulty'] for t in tracks]

In [ ]:
plt.figure()
pd.Series(difficulty).plot()

In [ ]:
plt.figure()
tracks.plot()

In [ ]:
closest

In [ ]:
from scipy.signal import butter, filtfilt

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y

In [ ]:
data[acc].apply(lambda x: butter_lowpass_filter(x, 3, 204.8)).plot()

In [ ]:
fig, axs = plt.subplots(3, sharex=True, figsize=(8, 10))

data[acc].rolling(600).median().plot(ax=axs[0])
gyro_filtered = data[gyro].rolling(600).mean()
gyro_filtered.plot(ax=axs[1])
height = calc_height(data[['baro']], T, ref_baro, ref_height)
calc_height(data[['baro']], T, ref_baro, ref_height).plot(ax=axs[2])

plt.tight_layout()

In [ ]:
from scipy.signal import savgol_filter

def baro_segment(height_data):
    filtered = height_data.rolling(1000).median().diff().rolling(2000).sum()
    segments = pd.Series(np.array([0] * len(height)))
    segments.index = filtered.index
    segments[(filtered > 1).values] = 1
    segments[(filtered < -2).values] = -1
    return segments

In [ ]:
from bokeh.models import BoxAnnotation

p = figure(plot_width=800, plot_height=800)
p.line(height)
BoxAnnotation

In [ ]:
plt.figure()
plt.plot(baro_segment(height['baro']))
((height/1000) - 1.5).plot(ax=plt.gca())

In [ ]:
def find_active_regions(data):
    # Must be moving downwards:
    return (height.rolling(1000).median().diff().rolling(2000).sum() < -1).fillna(False)

In [ ]:
gyro_active_filtered = gyro_filtered[find_active_regions(data).values]

In [ ]:
gyro_active_filtered.plot()

In [ ]:
fig, axs = plt.subplots(3, sharex=True, figsize=(8, 10))

data_active[acc].rolling(600).median().plot(ax=axs[0])
data_active[gyro].rolling(600).median().plot(ax=axs[1])
calc_height(data_active[['baro']], T, ref_baro, ref_height).plot(ax=axs[2])

plt.tight_layout()

In [ ]:
d = np.where(np.diff(np.sign(data['gyro_z'].rolling(600).median())))[0]
plt.figure()
plt.plot(d, [0]*len(d), 'o') 

In [ ]:
from slither.imu.orientation import MadgewickAHRS

In [ ]:
ori = MadgewickAHRS(sampling_rate=204.8)
ori = ori.update_series(np.deg2rad((data[gyro] - data[gyro].iloc[:2000].mean(axis=0)).values), data[acc].values)

In [ ]:
plt.figure()
plt.plot(ori)